In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install bing-image-downloader gradio tensorflow


In [ ]:
from bing_image_downloader import downloader

classes_roses = ["Rose Hybrid Tea", "Rose Floribunda", "Rose Grandiflora", "Rose Climbing", "Rose Miniature"]

for c in classes_roses:
    downloader.download(c, limit=40, output_dir='rose_dataset', adult_filter_off=True, force_replace=False, timeout=60)


[%] Downloading Images to /content/rose_dataset/Rose Hybrid Tea


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://cdn.commercev3.net/cdn.edmundsroses.com/images/popup/Rose_EnchantedPeace_Star_WEB_23682.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://cdn.commercev3.net/cdn.edmundsroses.com/images/popup/24080_b.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://fthmb.tqn.com/-32UwWL8XQtBjPz0dr2BVnsXJNs=/960x0/filters:no_upscale():max_bytes(150000):strip_icc()/126025681-56a347a63df78cf7727cad37.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://songofroses.com/wp-content/uploads/2023/05/Hybrid-Tea-Rose-Bush.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://www.thespruce.com/thmb/uBVIr54z-x8WcSte84xwIa4RIGc=/2742x0/filters:no_upscale():max_bytes(150000):strip_icc()/Rosa_Peace_1945-5b29cee4ba617700546dd09a.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://cdn.commercev3.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import pickle

img_size = (128,128)

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    "rose_dataset",
    target_size=img_size,
    batch_size=16,
    subset="training",
    class_mode="categorical"
)

val_gen = datagen.flow_from_directory(
    "rose_dataset",
    target_size=img_size,
    batch_size=16,
    subset="validation",
    class_mode="categorical"
)

model_rose = models.Sequential([
    layers.Flatten(input_shape=(128,128,3)),
    layers.Dense(512, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(len(classes_roses), activation="softmax")
])

model_rose.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model_rose.fit(train_gen, validation_data=val_gen, epochs=10)

# Lưu model và labels
model_rose.save("/content/drive/MyDrive/rose_ann_best.h5")
with open("/content/drive/MyDrive/rose_labels.pkl", "wb") as f:
    pickle.dump(list(train_gen.class_indices.keys()), f)

print("✅ Đã train và lưu model + labels vào Drive")


Found 160 images belonging to 5 classes.
Found 39 images belonging to 5 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.2874 - loss: 22.5150 - val_accuracy: 0.2308 - val_loss: 18.6588
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 872ms/step - accuracy: 0.2328 - loss: 17.3742 - val_accuracy: 0.1795 - val_loss: 7.5043
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 995ms/step - accuracy: 0.3172 - loss: 13.9750 - val_accuracy: 0.2564 - val_loss: 8.6510
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.2668 - loss: 8.9721 - val_accuracy: 0.2308 - val_loss: 5.9790
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 839ms/step - accuracy: 0.3956 - loss: 5.5797 - val_accuracy: 0.3333 - val_loss: 4.4396
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 987ms/step - accuracy: 0.4055 - loss: 3.8652 - val_accuracy: 0.2564 - val_loss: 3.0812
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.4264 - loss: 2.2509 - val_accuracy: 0.2051 - val_loss: 3.0578
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 901ms/step - accuracy: 0.4386 - loss: 1.9031 - val_accuracy: 0.

✅ Đã train và lưu model + labels vào Drive


In [ ]:
from tensorflow.keras.models import load_model
import pickle

model_rose = load_model("/content/drive/MyDrive/rose_ann_best.h5")

with open("/content/drive/MyDrive/rose_labels.pkl", "rb") as f:
    labels_roses = pickle.load(f)

print("✅ Đã load model và labels:", labels_roses)


✅ Đã load model và labels: ['Rose Climbing', 'Rose Floribunda', 'Rose Grandiflora', 'Rose Hybrid Tea', 'Rose Miniature']


In [ ]:
import numpy as np

def predict_rose(img):
    img = img.resize(img_size)
    x = np.array(img)/255.0
    x = np.expand_dims(x, axis=0)
    preds = model_rose.predict(x)
    return {labels_roses[i]: float(preds[0][i]) for i in range(len(labels_roses))}


In [ ]:
import os

feedback_dir = "/content/drive/MyDrive/rose_feedback"
os.makedirs(feedback_dir, exist_ok=True)

def save_feedback(img, correct_label):
    if correct_label not in labels_roses:
        return f"❌ Nhãn {correct_label} không hợp lệ. Các nhãn hợp lệ: {labels_roses}"

    label_dir = os.path.join(feedback_dir, correct_label)
    os.makedirs(label_dir, exist_ok=True)
    img_path = os.path.join(label_dir, f"fb_{len(os.listdir(label_dir))}.jpg")
    img.save(img_path)
    return f"✅ Feedback đã lưu với nhãn {correct_label}"

def update_model():
    datagen_fb = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    fb_gen = datagen_fb.flow_from_directory(
        feedback_dir,
        target_size=img_size,
        batch_size=8,
        class_mode="categorical"
    )

    if fb_gen.samples > 0:
        model_rose.fit(fb_gen, epochs=2)
        model_rose.save("/content/drive/MyDrive/rose_ann_best.h5")
        return "✅ Mô hình đã được cập nhật từ feedback."
    else:
        return "⚠️ Chưa có feedback để cập nhật."


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🌹 Nhận diện 5 giống hoa hồng + Feedback")

    with gr.Row():
        with gr.Column():
            img_input = gr.Image(type="pil", label="Upload ảnh hoa hồng")
            predict_btn = gr.Button("Dự đoán")
            output = gr.Label(num_top_classes=3)

        with gr.Column():
            correct_label = gr.Textbox(label="Nếu sai, nhập nhãn đúng (ví dụ: Rose Floribunda)")
            feedback_btn = gr.Button("Lưu Feedback")
            update_btn = gr.Button("Cập nhật mô hình")
            fb_msg = gr.Textbox(label="Thông báo")

    predict_btn.click(predict_rose, inputs=img_input, outputs=output)
    feedback_btn.click(save_feedback, inputs=[img_input, correct_label], outputs=fb_msg)
    update_btn.click(lambda: update_model(), outputs=fb_msg)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5e60ae4d23dd42077e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
